# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 8:30AM,256855,10,Enova-11326,Emerginnova,Released
1,Feb 8 2023 8:30AM,256855,10,Enova-11327,Emerginnova,Released
2,Feb 8 2023 8:30AM,256855,10,Enova-11328,Emerginnova,Released
3,Feb 8 2023 8:30AM,256854,10,130608,"Virtue Rx, LLC",Released
4,Feb 7 2023 2:51PM,256851,10,7564087,"SD Head USA, LLC",Released
5,Feb 7 2023 2:39PM,256850,10,SOTAH0000900,"Nextsource Biotechnology, LLC",Released
6,Feb 7 2023 2:39PM,256850,10,SONSB0002099,"Nextsource Biotechnology, LLC",Released
7,Feb 7 2023 2:39PM,256850,10,SONSB0002100,"Nextsource Biotechnology, LLC",Released
8,Feb 7 2023 2:38PM,256849,10,Yusen-9128698,Yusen – 3D Matrix,Executing
9,Feb 7 2023 2:38PM,256849,10,Yusen-9128717,Yusen – 3D Matrix,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,256849,Released,2
41,256850,Released,3
42,256851,Released,1
43,256854,Released,1
44,256855,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256849,NaN,4.0,2.0
256850,NaN,NaN,3.0
256851,NaN,NaN,1.0
256854,NaN,NaN,1.0
256855,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0.0,0.0,1.0
256782,55.0,29.0,21.0
256788,0.0,0.0,1.0
256792,8.0,5.0,2.0
256797,0.0,0.0,38.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0,0,1
256782,55,29,21
256788,0,0,1
256792,8,5,2
256797,0,0,38


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,0,0,1
1,256782,55,29,21
2,256788,0,0,1
3,256792,8,5,2
4,256797,0,0,38


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,,,1
1,256782,55,29,21
2,256788,,,1
3,256792,8,5,2
4,256797,,,38


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 8:30AM,256855,10,Emerginnova
3,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC"
4,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC"
5,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC"
8,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix
14,Feb 7 2023 2:34PM,256848,10,ISDIN Corporation
21,Feb 7 2023 2:26PM,256847,10,Emerginnova
22,Feb 7 2023 2:19PM,256846,10,"Methapharm, Inc."
23,Feb 7 2023 2:07PM,256845,21,"NBTY Global, Inc."
24,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 8:30AM,256855,10,Emerginnova,,,3
1,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC",,,1
2,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC",,,1
3,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC",,,3
4,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix,,4,2
5,Feb 7 2023 2:34PM,256848,10,ISDIN Corporation,,,7
6,Feb 7 2023 2:26PM,256847,10,Emerginnova,,,1
7,Feb 7 2023 2:19PM,256846,10,"Methapharm, Inc.",,,1
8,Feb 7 2023 2:07PM,256845,21,"NBTY Global, Inc.",,,1
9,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256855,10,Emerginnova,3,,
1,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC",1,,
2,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC",1,,
3,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC",3,,
4,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix,2,4,
5,Feb 7 2023 2:34PM,256848,10,ISDIN Corporation,7,,
6,Feb 7 2023 2:26PM,256847,10,Emerginnova,1,,
7,Feb 7 2023 2:19PM,256846,10,"Methapharm, Inc.",1,,
8,Feb 7 2023 2:07PM,256845,21,"NBTY Global, Inc.",1,,
9,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",13,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256855,10,Emerginnova,3,,
1,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC",1,,
2,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC",1,,
3,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC",3,,
4,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix,2,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256855,10,Emerginnova,3.0,NaN,NaN
1,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC",1.0,NaN,NaN
2,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC",1.0,NaN,NaN
3,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
4,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix,2.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256855,10,Emerginnova,3.0,0.0,0.0
1,Feb 8 2023 8:30AM,256854,10,"Virtue Rx, LLC",1.0,0.0,0.0
2,Feb 7 2023 2:51PM,256851,10,"SD Head USA, LLC",1.0,0.0,0.0
3,Feb 7 2023 2:39PM,256850,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
4,Feb 7 2023 2:38PM,256849,10,Yusen – 3D Matrix,2.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6934402,211.0,9.0,8.0
12,256841,0.0,4.0,0.0
15,256782,21.0,29.0,55.0
16,513639,1.0,1.0,0.0
19,513652,9.0,2.0,0.0
20,513613,2.0,0.0,0.0
21,1027300,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6934402,211.0,9.0,8.0
1,12,256841,0.0,4.0,0.0
2,15,256782,21.0,29.0,55.0
3,16,513639,1.0,1.0,0.0
4,19,513652,9.0,2.0,0.0
5,20,513613,2.0,0.0,0.0
6,21,1027300,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,211.0,9.0,8.0
1,12,0.0,4.0,0.0
2,15,21.0,29.0,55.0
3,16,1.0,1.0,0.0
4,19,9.0,2.0,0.0
5,20,2.0,0.0,0.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,211.0
1,12,Released,0.0
2,15,Released,21.0
3,16,Released,1.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,8.0,0.0,55.0,0.0,0.0,0.0,0.0
Executing,9.0,4.0,29.0,1.0,2.0,0.0,0.0
Released,211.0,0.0,21.0,1.0,9.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,55.0,0.0,0.0,0.0,0.0
1,Executing,9.0,4.0,29.0,1.0,2.0,0.0,0.0
2,Released,211.0,0.0,21.0,1.0,9.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,55.0,0.0,0.0,0.0,0.0
1,Executing,9.0,4.0,29.0,1.0,2.0,0.0,0.0
2,Released,211.0,0.0,21.0,1.0,9.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()